# Dimensionality reduction for image classification

#### Goal

In some of the previous notebooks, we showed how dimensionality reduction allows us to visualize our data. Principal Components Analysis, or PCA, gives us a way to rotate the axes and write our data using much fewer dimensions. 

In our last notebook on PCA, we tried to apply PCA in a naive way to the MNIST dataset, but in this notebook we'll do something more clever!

We will investigate in this notebook how PCA can be used for image classification.

#### Strategy

Recall that the basic idea of PCA was to find the axis rotations that goes through our data, as demonstrated by this figure:

![PCA](https://newonlinecourses.science.psu.edu/stat857/sites/onlinecourses.science.psu.edu.stat857/files/lesson05/PCA_plot/index.gif)

Now let's take that one step further. If we perform a PCA on the subset of the data that corresponds to images of the digit `1`, then the axis that is found will do well at reproducing data from pictures of 1's, but not other digits. 

Therefore, we should be able to do the following:

* perform PCA/find the principal components for the subset of the data pertaining to each image class (the subset of data associated with each unique label)
* transform and reconstruct new, unlabeled images, with each of the sets of principal components/each of the PCA output models trained on each image class
* classify each new image by identifying the PCA model that does the best job of reconstructing the original image from a transformation.

## Data Setup

First let's pull in the MNIST data.

In [ ]:
using Flux, Flux.Data.MNIST, Images

imgs = MNIST.images()
labels = MNIST.labels();

Here's what the first 24 images look like:

In [ ]:
display(reshape(imgs[1:24],8,3))
println(labels[1:24]')

There are 60,000 of these images, each with 784 pixels. This means our dataset is of size `784 x 60000`, and our data is 784-dimensional.

In [ ]:
matrix =  hcat([vec(Float64.(i)) for i in imgs]...);
size(matrix)

## Dimensionality reduction classification

First let's generate training data. The command we use is as follows:

In [ ]:
training_size  = 500
singular_vec_size = 25
training_data = [ matrix[:,find(labels.== (i%10))[1:training_size]] for i=0:9];

This makes `training_data[i]` an array of 500 samples for the `i`th digit, i.e. `training_data[5]` is 500 images of the digit `5`. 

Train one PCA model on each of these data subsets (on each of the arrays contained in `training_data`). Store the models in an output called `Ms`.

Note: for simplicity, set a maximum output dimension as 25 when you fit PCA.

#### Solution

Now we have that `Ms[i]` is the PCA model for the `i`th digit. Now let's calculate how well each PCA does on an image. The way we do this is to take a data point (a column of `matrix`). Then, for each PCA model

1. Transform to the PC axis (dimensionality reduce)
2. Reconstruct the original data point by reversing the dimensionality reduction
3. Calculate the difference between the original data point and the reconstructed data point

At this point, you should be able to classify the image associated with the original data point by identifying the PCA model that does the best at reconstructing it.

Write a function that takes the dataset (`matrix`), chooses a random image from it, and returns both the image's label and its classification. 

#### Solution

## Conclusion

One way to think about dimensionality reduction is as information retention. Dimensionality reduction models try to retain as much information as possible with as few numbers as possible. Different pieces of the dataset utilize different numbers, and thus the way you would want to dimensionally reduce is dependent on what you want to store. For the pictures of apples, the mean of the red pixels and the height of the apple might be most of the information, while for a banana the height may not be a differentiating factor. Thus by seeing how the dimensionality reduction is able to reproduce the image, we are able to back out what image we are looking at!

## Mathematical bonus (Optional, for the mathematically-inclined!)

Above we used statistics packages for dimensionality reduction, but we could do this directly by using the 
**singular value decomposition** (SVD). This is the mathematical operation of finding a matrix factorization `A = U S V'`, where `U`'s leading columns are a low rank approximation to `A`. Thus we can generate projection matrices using this matrix from the factorization:

In [ ]:
u = [svdfact(training_data[i])[:U][:,1:singular_vec_size] for i=1:10];

And then we can classify based on the reconstruction error using this low-rank approximation

In [ ]:
z = vcat([sum((u[i]'matrix).^2,1) for i=1:10]...)
result = rem.( [ findmax(z[:,i])[2] for i=1:60000] ,10)
mean(result .== labels)

Notice that we get the same result. This shows that, internally, PCA is the dimensionality reduction algorithm which is simply using the low-rank approximation of an SVD factorization!